In [2]:
!pip install PyQt5

  Obtaining dependency information for PyQt5 from https://files.pythonhosted.org/packages/ca/ac/596e8ca16fd0634542d874c0d79219fc527ea7de73a5000092f60ecbf6e9/PyQt5-5.15.10-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for PyQt5-sip<13,>=12.13 from https://files.pythonhosted.org/packages/8e/e2/d296cb17bae19ba49137a2649934a70d5f48cc99b7daa6ce9cf1aecddfeb/PyQt5_sip-12.13.0-cp311-cp311-win_amd64.whl.metadata
  Using cached PyQt5_Qt5-5.15.2-py3-none-win_amd64.whl (50.1 MB)
Using cached PyQt5-5.15.10-cp37-abi3-win_amd64.whl (6.8 MB)
Using cached PyQt5_sip-12.13.0-cp311-cp311-win_amd64.whl (78 kB)



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib qt
import seaborn as sns
import os
from os import path
import datetime
import xlrd
plt.style.use('seaborn-v0_8-ticks')
from scipy.signal import find_peaks
import itertools
from pylab import *
import csv

Pobieranie zestawu danych

In [ ]:
folder_path_15 = "C:\\Users\\48503\\Desktop\\joint_symbolic\\controlled_breathing"
folders_15 = os.listdir(folder_path_15)
breathing_dict_15 = {folder : {} for folder in folders_15}

for folder_name in folders_15:
    path = os.path.join(folder_path_15, folder_name)
    for file_name in os.listdir(path):
        if file_name.endswith('.csv'):
         df_name = file_name.split('_ODD')[0]
         df = pd.read_csv(os.path.join(path, file_name))
         breathing_dict_15[folder_name][df_name] = df

Pobranie zestawu danych, gdzie występują dwa typy plików csv

In [3]:
folder_path_56 = "C:\\Users\\48503\\Desktop\\joint_symbolic\\controlled_breathing_full2"
folders_56 = os.listdir(folder_path_56)
breathing_dict_56 = {folder : {} for folder in folders_56}

for folder_name in folders_56:
    folder_path = os.path.join(folder_path_56, folder_name)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
          df_name = file_name.split('PAC_')[1].split('_')[0] #id to numer pacjenta pobrany z nazwy pliku
          file_path = os.path.join(folder_path, file_name)
          csv_file = open(file_path, 'r')
          try:
              dialect = csv.Sniffer().sniff(csv_file.readline(), ';')   
          except:
              dialect = csv.Sniffer().sniff(csv_file.readline(), ',')   
          csv_file.seek(0)
          df = pd.read_csv(file_path, on_bad_lines='skip', sep=dialect.delimiter)
          if dialect.delimiter == ";":
              for row in df:
                  df[row] = df[row].str.replace(',','.').astype(float)
          breathing_dict_56[folder_name][df_name] = df

Pobranie danych z bazy 37 plików, gdzie id pacjenta to numer z nazwy folderu "ochotnik"; można też pobrać id jako numer z nazwy folderu + 56 w przypadku integracji obu baz (df_name = str(int(folder_name.split('OCHOTNIK_')[1].split('_')[0]) + 56)), żeby numery id się nie powtarzały

In [35]:
folder_path_37 = "C:\\Users\\48503\\Desktop\\joint_symbolic\\CONTROL_BREATHING_RECORDINGS"
breathing_dict_keys = ['6_breaths', '10_breaths', '15_breaths', 'baseline']
breathing_dict_37 = {frequency:{} for frequency in breathing_dict_keys}
folders_37 = os.listdir(folder_path_37)

for folder_name in folders_37:
    folder_path = os.path.join(folder_path_37, folder_name)
    df_number = folder_name.split('OCHOTNIK_')[1].split('_')[0] 
    for next_folder_name in os.listdir(folder_path):
        try:
           breathing_frequency = next_folder_name.split('CLEAN_DATA')[1].lower()[1:]
        except:
            breathing_frequency = "RAW_DATA"
        if breathing_frequency in breathing_dict_37:
            next_folder_path = os.path.join(folder_path, next_folder_name)
            for file_name in os.listdir(next_folder_path):
                if file_name.endswith('.csv'): 
                    file_path = os.path.join(next_folder_path, file_name)
                    csv_file = open(file_path, 'r')
                    try:
                        dialect = csv.Sniffer().sniff(csv_file.readline(), ';')   
                    except:
                        dialect = csv.Sniffer().sniff(csv_file.readline(), ',')      
                    csv_file.seek(0)
                    decimal_map = {";" : ",", "," : "."}
                    my_df = pd.read_csv(file_path, encoding='latin-1', decimal=decimal_map[dialect.delimiter], on_bad_lines='skip', sep=dialect.delimiter)
                    breathing_dict_37[breathing_frequency][df_number] = my_df

C:\Users\48503\AppData\Local\Temp\ipykernel_21888\2752246935.py:26: DtypeWarning: Columns (3,13) have mixed types. Specify dtype option on import or set low_memory=False.
  my_df = pd.read_csv(file_path, encoding='latin-1', decimal=decimal_map[dialect.delimiter], on_bad_lines='skip', sep=dialect.delimiter)


Jeden plik ma błędy w wierszach powyżej indeksu 26966 oraz mieszane kolumny, żeby przerobić kolumny typu object (z przecinkami) na float (z kropkami) trzeba usunąć błędne wiersze (w podglądzie są one i tak puste):

In [ ]:
breathing_dict_37["10_breaths"]["3"].iloc[26966]

In [36]:
breathing_dict_37["10_breaths"]["3"] = breathing_dict_37["10_breaths"]["3"][breathing_dict_37["10_breaths"]["3"].index < 26967] 
breathing_dict_37["10_breaths"]["3"] = breathing_dict_37["10_breaths"]["3"].drop('hbr_s1_d1[umol/L]', axis=1)
obj_cols = breathing_dict_37["10_breaths"]["3"].select_dtypes(include=['object']).apply(lambda col: col.apply(lambda item: float(item.replace(',', '.'))))
breathing_dict_37["10_breaths"]["3"][obj_cols.columns] = obj_cols

In [ ]:
breathing_dict_37["10_breaths"]["3"].tail()

Połączenie bazy 37 i 56 plików w jedną

Sprawdzenie nazw kolumn w plikach

In [38]:
def check_column_names(breathing_dict):
    for breath_frequency in breathing_dict:
        for patient_nr, patient_df in breathing_dict[breath_frequency].items(): 
            print(patient_df.columns)                

In [ ]:
check_column_names(breathing_dict_56)

In [ ]:
check_column_names(breathing_dict_37)

In [40]:
def how_many_have_ecg(breathing_dict):
    for breath_frequency in breathing_dict:
        ecg_number = 0
        patients_with_ecg = []
        for patient_nr, patient_df in breathing_dict[breath_frequency].items(): 
            if "ecg" in patient_df.columns:
                ecg_number += 1
                patients_with_ecg.append(patient_nr)
        print(f"Liczba osób z ECG dla częstotliwości {breath_frequency}: {ecg_number}")  
        print(f"Numery osób z ECG dla częstotliwości {breath_frequency}: {patients_with_ecg} \n")

In [ ]:
how_many_have_ecg(breathing_dict_37)

Zamiana nazw kolumn na jednolite
"fv_r[]", "fv_r[fv_r]" -> "fv_r"
"fv_l[]", "fv_r[fv_l]" -> "fv_l"
"ekg[]", "ekg[ekg]" -> "ekg"
"abp_arm[mm_Hg]", "abp_arm[abp_arm]" -> "abp_arm"

In [42]:
def unify_column_names(breathing_dict):
    column_names_map = {"fv_r[]":"fv_r", "fv_r[fv_r]":"fv_r", "fvr":"fv_r", "fv_l[]":"fv_l", 
                    "fv_l[fv_l]":"fv_l", "fvl":"fv_l", "ekg[]":"ekg", "ekg[ekg]":"ekg", "ecg":"ekg", "abp_arm[mm_Hg]":"abp_arm", "abp_arm[abp_arm]":"abp_arm", "abp_cnap[mmHg]":"abp_arm", "etco2[mmHg]":"etco2"}
    for breath_frequency in breathing_dict:
        for patient_nr, patient_df in breathing_dict[breath_frequency].items(): 
            patient_df.rename(columns=column_names_map, inplace=True)

In [43]:
unify_column_names(breathing_dict_37)

Sprawdzenie nazw kolumn w plikach po raz drugi

In [ ]:
check_column_names(breathing_dict_56)

In [ ]:
check_column_names(breathing_dict_37)

Funkcja konwertująca czas

In [45]:
def icmp_dateformat_to_datetime(icmp_time_mark):
    datetime_date = xlrd.xldate_as_datetime(icmp_time_mark, 0)
    datetime_date = datetime_date + datetime.timedelta(hours=1)
    return datetime_date

Sprawdzenie procentu brakujących wartości w kolumnach z ecg, abp, fv oraz etco2

In [46]:
def print_nan_percent(breathing_dict):
    nan_dict = {breath_frequency : {} for breath_frequency in breathing_dict}
    for breath_frequency in breathing_dict:
        for patient_nr, patient_df in breathing_dict[breath_frequency].items():
            nan_string = ""
            for col_name in ['abp_arm', 'ekg', 'fv_r', 'fv_l', 'etco2']:
                if col_name in patient_df.columns:
                    nan_percent = patient_df[col_name].isnull().sum() * 100 / len(patient_df)
                    if nan_percent != 0.0:
                       nan_string += col_name + " : " + str(round(nan_percent, 2)) + "; "
            nan_dict[breath_frequency][patient_nr] = nan_string
    nan_df = pd.DataFrame(nan_dict)
    return nan_df

In [ ]:
print_nan_percent(breathing_dict_56)

In [47]:
print_nan_percent(breathing_dict_37)

,6_breaths,10_breaths,15_breaths,baseline
1,,,,
10,,NaN,,
12,,,,
13,,,,
14,,fv_r : 0.13; fv_l : 0.13; etco2 : 0.2;,,
15,etco2 : 21.43;,etco2 : 52.85;,,
16,,,,
17,,,,
18,,,,
19,,,,


In [48]:
def delete_keys(to_delete, breathing_dict):
    for breath_freq in breathing_dict:
        for key_to_delete in to_delete:
            breathing_dict[breath_freq].pop(key_to_delete, None)

Usuwanie ochotnika numer 35 z bazy 37 (zła jakość sygnału) oraz ochotnika numer 10

In [49]:
to_delete_37 = ["35", "10"]
delete_keys(to_delete_37, breathing_dict_37)

Dodanie kolumny z datą i godziną do każdego dataframe i wypełnienie braków danych (NaN) wartościami średnimi

In [50]:
def fill_nan_and_add_time(breathing_dict):    
    for breathing_freq in breathing_dict:
        for patient_number, patient_df in breathing_dict[breathing_freq].items():
            print(patient_number +": "+breathing_freq)
            patient_df['DateFormat'] = patient_df['DateTime'].map(icmp_dateformat_to_datetime)
            patient_df.fillna(patient_df.mean(), inplace=True)

In [ ]:
fill_nan_and_add_time(breathing_dict_56)

In [51]:
fill_nan_and_add_time(breathing_dict_37)

1: 6_breaths
12: 6_breaths
13: 6_breaths
14: 6_breaths
15: 6_breaths
16: 6_breaths
17: 6_breaths
18: 6_breaths
19: 6_breaths
2: 6_breaths
20: 6_breaths
21: 6_breaths
22: 6_breaths
23: 6_breaths
24: 6_breaths
25: 6_breaths
26: 6_breaths
27: 6_breaths
28: 6_breaths
29: 6_breaths
3: 6_breaths
30: 6_breaths
31: 6_breaths
32: 6_breaths
33: 6_breaths
34: 6_breaths
36: 6_breaths
37: 6_breaths
4: 6_breaths
5: 6_breaths
6: 6_breaths
7: 6_breaths
8: 6_breaths
9: 6_breaths
1: 10_breaths
12: 10_breaths
13: 10_breaths
14: 10_breaths
15: 10_breaths
16: 10_breaths
17: 10_breaths
18: 10_breaths
19: 10_breaths
2: 10_breaths
20: 10_breaths
21: 10_breaths
22: 10_breaths
23: 10_breaths
24: 10_breaths
25: 10_breaths
26: 10_breaths
27: 10_breaths
28: 10_breaths
29: 10_breaths
3: 10_breaths
30: 10_breaths
31: 10_breaths
32: 10_breaths
33: 10_breaths
34: 10_breaths
36: 10_breaths
37: 10_breaths
4: 10_breaths
5: 10_breaths
6: 10_breaths
7: 10_breaths
8: 10_breaths
9: 10_breaths
1: 15_breaths
12: 15_breaths
13:

In [59]:
print_nan_percent(breathing_dict_37)

,6_breaths,10_breaths,15_breaths,baseline
1,,,,
12,,,,
13,,,,
14,,,,
15,,,,
16,,,,
17,,,,
18,,,,
19,,,,
2,,,,


Funkcja znajdująca piki, diff to jest RR w przypadku ECG tylko trzeba je przemnożyć przez 0.005 (1/(200 Hz)), peaks to numery próbek odpowiadających pikom, signal[peaks] to wartości sygnału odpowiadające tym numerom próbek

In [52]:
def get_peaks(signal):
    peaks, _ = find_peaks(signal, distance=115)
    difference = np.diff(peaks)  
    return peaks, signal[peaks], difference

Wyrysowanie 600 pierwszych próbek każdego sygnału z zaznaczonymi znalezionymi pikami ("x")

In [53]:
def check_peak_detection_accuracy(plot_folder_path, data_dict):
    for breath_frequency in data_dict:
        for dataframe_nr, dataframe in data_dict[breath_frequency].items():       
            fig, ax = plt.subplots(4, 1, tight_layout = True, figsize=(16, 32)) 
            for i, col_name in enumerate(['abp_arm', 'ekg', 'fv_l', 'fv_r']):
                if col_name in dataframe.columns:
                    x = dataframe[col_name].iloc[:600]
                    ax[i].plot(x)
                    my_peaks = get_peaks(x)
                    ax[i].plot(my_peaks[0], my_peaks[1], "x")
                    ax[i].set_ylabel(f'{col_name}', fontsize=25)
                    ax[i].tick_params(axis='both', labelsize=18)
                    ax[i].grid(True)
            ax[3].set_xlabel("nr probki", fontsize = 28)
            savefig_path = plot_folder_path+f"\\{breath_frequency}\\{dataframe_nr}_plot.pdf"
            plt.savefig(savefig_path, bbox_inches='tight', dpi=600)
            plt.show()
            plt.close()

In [15]:
folder_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\timeseries_plots\\56"
check_peak_detection_accuracy(folder_path, breathing_dict_56)

In [54]:
folder_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\timeseries_plots\\37"
check_peak_detection_accuracy(folder_path, breathing_dict_37)

Usuwanie uprzednio wykluczonych pacjentów numer 2, 6 i 33 oraz dodatkowo 14 i 31 (zła jakość sygnału) z bazy 56

In [89]:
to_delete_56 = ["2", "6", "33", "14", "31"]
delete_keys(to_delete_56, breathing_dict_56)

In [90]:
def make_empty_matrix():
    my_permutations = [''.join(map(str, triplet)) for triplet in list((itertools.product([1, 0], repeat=3)))]
    my_matrix = pd.DataFrame()
    for triplet_1 in my_permutations: 
       for triplet_2 in my_permutations:
            my_matrix.at["'"+triplet_1, "'"+triplet_2] = 0
    return my_matrix

In [91]:
def get_diff_df(dataframe, is_fv, is_left):
    sap = get_peaks(dataframe['abp_arm'])[1]
    if is_fv:
        sap = get_peaks(dataframe['fv_l'])[1]
        if not is_left:
            sap = get_peaks(dataframe['fv_r'])[1]
            
    try:        
        rr = get_peaks(dataframe['ekg'])[2]*0.005
    except:
        rr = get_peaks(dataframe['abp_arm'])[2]*0.005 #u osób bez EKG korzystamy z ABP
        
    sap_diff = np.diff(sap)
    rr_diff = np.diff(rr)
    diff_dataframe = pd.DataFrame(list(zip(sap_diff, rr_diff)), columns=['sap_diff', 'rr_diff'])
    return diff_dataframe

In [92]:
def make_and_save_heatmap(relative_array, breath_frequency, is_fv, heatmap_path):
    my_custom_palette = sns.color_palette("ch:-.24", as_cmap=True)
    my_heatmap = sns.heatmap(relative_array, cmap = my_custom_palette, annot=True)
    my_heatmap.xaxis.tick_top()
    my_heatmap.xaxis.set_ticks_position('none')
    fontsize = 10
    plt.xlabel('SAP', fontsize = fontsize) 
    my_heatmap.xaxis.set_label_position('top') 
    plt.ylabel('RR', fontsize = fontsize) 
    if is_fv:
        plt.savefig(heatmap_path+f"\\{breath_frequency}_heatmap_fv.pdf", bbox_inches='tight', dpi=600)
        relative_array.to_csv(heatmap_path+f"\\{breath_frequency}_matrix_fv.csv")
    else:
        plt.savefig(heatmap_path+f"\\{breath_frequency}_heatmap_abp.pdf", bbox_inches='tight', dpi=600)
        relative_array.to_csv(heatmap_path+f"\\{breath_frequency}_matrix_abp.csv")
    plt.show()
    plt.close()

In [93]:
def save_jsd(is_fv, jsd_df, jsd_path):
    if is_fv:
        jsd_df.T.to_csv(jsd_path+"\\jsd_fv.csv")
    else:    
        jsd_df.T.to_csv(jsd_path+"\\jsd_abp.csv")

In [97]:
def add_mean_params(jsd_df, dataframe, is_left, patient_number, freq):
    window_size = 2000
    for col_name in ['abp_arm', 'ekg', 'etco2']:
        if col_name in dataframe.columns:
            nan_percent = dataframe[col_name].isnull().sum() * 100 / len(dataframe)
            if nan_percent != 100.0:
                jsd_df.at["mean_"+col_name+"_"+freq, patient_number] = dataframe[col_name].rolling(window=window_size).mean().dropna().mean()
    if is_left:
        jsd_df.at["mean_fvr/l_"+freq, patient_number] = dataframe['fv_l'].rolling(window=window_size).mean().dropna().mean()
    else:
        jsd_df.at["mean_fvr/l_"+freq, patient_number] = dataframe['fv_r'].rolling(window=window_size).mean().dropna().mean()
    return jsd_df

In [98]:
def joint_symbolic(data_dict, is_fv, metadata_path, heatmap_path, jsd_path):
    jsd_df = pd.DataFrame()
    metadata = pd.read_csv(metadata_path, sep = ";")
    for freq in data_dict:
        freq_matrix = make_empty_matrix()
        for patient_number, dataframe in data_dict[freq].items():
            is_left = bool(metadata.at[metadata.loc[metadata['L.poj.'] == int(patient_number)].index[0], 'FV_LEWA_MCA'])
            patient_freq_matrix = make_empty_matrix()
            diff_df = get_diff_df(dataframe, is_fv, is_left)
            
            for col_name in ['sap_diff', 'rr_diff']:
                diff_df[f'{col_name}_binary'] = diff_df.apply(lambda df_row: 1 if df_row[col_name] > 0 else 0, axis = 1)
            
            for index in range(0, len(diff_df)):
                sap_triplet =  ''.join(diff_df['sap_diff_binary'].iloc[index:index+3].astype(str))
                rr_triplet =  ''.join(diff_df['rr_diff_binary'].iloc[index:index+3].astype(str))
                if len(rr_triplet) > 2 and len(sap_triplet) > 2:
                     freq_matrix.at["'"+rr_triplet, "'"+sap_triplet] += 1 #kreska potrzebna, żeby R nie usuwało początkowych zer
                     patient_freq_matrix.at["'"+rr_triplet, "'"+sap_triplet] += 1 
            
            relative_patient_matrix = patient_freq_matrix.div(np.sum(patient_freq_matrix.values))   
            signal_map = {True : 'FV_', False : 'ABP_'}
            jsd_df.at[signal_map[is_fv]+"jsd_sym_"+freq, patient_number] = np.sum(np.diag(relative_patient_matrix)) 
            jsd_df.at[signal_map[is_fv]+"jsd_diam_"+freq, patient_number] = np.sum(np.diag(np.rot90(relative_patient_matrix)))
            jsd_df = add_mean_params(jsd_df, dataframe, is_left, patient_number, freq)
            
        relative_freq_matrix = freq_matrix.div(np.sum(freq_matrix.values))        
        make_and_save_heatmap(relative_freq_matrix, freq, is_fv, heatmap_path)
    save_jsd(is_fv, jsd_df, jsd_path)     

In [69]:
jsa_path = "C:\\Users\\48503\\Desktop\\joint_symbolic\\"

Joint symbolic dla bazy 56

In [120]:
meta_path_56 = jsa_path + "dane_sonata\\dane_sonata_56.csv"
heatmap_path_56 = jsa_path + "heatmaps\\56"
jsd_path_56 = jsa_path + "jsd\\56"

In [98]:
joint_symbolic(breathing_dict_56, False, meta_path_56, heatmap_path_56, jsd_path_56) #abp
joint_symbolic(breathing_dict_56, True, meta_path_56, heatmap_path_56, jsd_path_56)  #fv

Joint symbolic dla bazy 37

In [88]:
meta_path_37 = jsa_path + "dane_sonata\\dane_sonata_37.csv"
heatmap_path_37 = jsa_path + "heatmaps\\37"
jsd_path_37 = jsa_path + "jsd\\37"

In [112]:
joint_symbolic(breathing_dict_37, False, meta_path_37, heatmap_path_37, jsd_path_37) #abp
joint_symbolic(breathing_dict_37, True, meta_path_37, heatmap_path_37, jsd_path_37)  #fv

Sklejanie plików z jsd i średnimi dla fv i abp 

In [133]:
jsd_path_37 = jsa_path + "jsd\\37\\"
jsd_abp = pd.read_csv(jsd_path_37+'jsd_abp.csv', index_col='Unnamed: 0')
jsd_fv = pd.read_csv(jsd_path_37+'jsd_fv.csv', index_col='Unnamed: 0')

jsd_fv = jsd_fv.drop(columns=jsd_abp.columns.intersection(jsd_fv.columns)) #usuwam powtarzające się kolumny

jsd = jsd_abp.merge(jsd_fv, left_index=True, right_index=True, how='inner')
jsd_sorted = jsd.sort_index()
jsd_sorted.to_csv(jsd_path_37+'jsd_all.csv', index_label='patient_id')

Zmiana formy macierzy z relatywną częstotliwością na zwykłą tabelkę w celu narysowania histogramów 3D w R

In [129]:
def transform_matrix(heatmap_path):
    save_path = heatmap_path + "\\transformed\\"
    for my_file_name in os.listdir(heatmap_path):
        if my_file_name.endswith('.csv'):
           dataframe = pd.read_csv(heatmap_path + f"\\{my_file_name}", index_col='Unnamed: 0')
           new_df = pd.DataFrame(columns=['RR', 'SAP', 'relative frequency'])

           for idx in dataframe.index:
              for col in dataframe.columns:
                 new_df = pd.concat([new_df, pd.DataFrame({'RR': [idx], 'SAP': [col], 'relative frequency': [dataframe.at[idx, col]]})], ignore_index=True)
           new_df.to_csv(save_path + f"\\{my_file_name}", index=False)

In [130]:
transform_matrix(heatmap_path_37)

C:\Users\48503\AppData\Local\Temp\ipykernel_21888\4038159935.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame({'RR': [idx], 'SAP': [col], 'relative frequency': [dataframe.at[idx, col]]})], ignore_index=True)
C:\Users\48503\AppData\Local\Temp\ipykernel_21888\4038159935.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame({'RR': [idx], 'SAP': [col], 'relative frequency': [dataframe.at[idx, col]]})], ignore_index=